In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType


spark = SparkSession.builder.master("local").appName('Read Something').getOrCreate()

schema_data = StructType([
                          StructField("Character", StringType(), False),
                          StructField("Movie", IntegerType(), False)
                        ])


In [48]:
df_movies = spark.read.csv('/home/benito.ramos/Desktop/GIT/PySpark/inputs/data_movies',schema_data, header=False )


In [64]:
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

df = df_movies.withColumn("index",row_number().over(Window.orderBy(monotonically_increasing_id()))
)


In [66]:
df.registerTempTable('StarWars')

/home/benito.ramos/.local/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [78]:
from pyspark.sql import SQLContext

df_movies = SQLContext.createDataFrame('/home/benito.ramos/Desktop/GIT/PySpark/inputs/data_movies',schema_data)



AttributeError: 'str' object has no attribute 'sparkSession'

In [76]:
sqlContext.sql(""" SELECT * FROM STARWARS """).show()

NameError: name 'sqlContext' is not defined